<a href="https://colab.research.google.com/github/dvschultz/stylegan_xl/blob/main/SG_XL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN-XL

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 imageio-ffmpeg

In [ ]:
import os
!git clone https://github.com/dvschultz/stylegan_xl

if(os.path.isdir('/content/drive/MyDrive/stylegan_xl')):
    %cd /content/drive/MyDrive/stylegan_xl
else:
    %cd /content/drive/MyDrive
    !git clone https://github.com/dvschultz/stylegan_xl
    %cd stylegan_xl/


## Generate dataset at every scale

Set the `source` to the path to your unzipped images.
Set `sizes` to the scales needed (you should always keep `16`, `32`, `64`, `128`, and `256`.)

In [ ]:
%cd /content
!unzip /content/drive/MyDrive/datasets/ladiescrop-1024.zip
%cd /content/stylegan_xl

In [ ]:
!python dataset_tool.py --help

In [ ]:
source = "/content/drive/MyDrive/weinrich.zip"
sizes = [16,32,64,128,256,512,1024]

for size in sizes:
    !python dataset_tool.py --source={source} --dest=/content/drive/MyDrive/stylegan_xl/data/weinrich-{size}.zip \
        --resolution={size}x{size} --transform=center-crop

## Train 16x16 model

*   For V100, use `batch-gpu=8`
*   A100s can likely use `batch-gpu=16`




In [ ]:
!python train.py --help

In [ ]:
!python /content/stylegan_xl/train.py \
--outdir=/content/drive/MyDrive/stylegan_xl/training-runs/weinrich \
--cfg=stylegan3-t \
--data=/content/drive/MyDrive/stylegan_xl/data/weinrich-16.zip \
--gpus=1 \
--batch=128 \
--mirror=1 \
--snap=8 \
--batch-gpu=8 \
--kimg=5000 \
--syn_layers=10

## Train upscaling models

*   `32` resolution: `--batch-gpu 8` on V100, `--batch-gpu 16` on A100
*   `64` resolution: `--batch-gpu 8` on V100, `--batch-gpu 16` on A100
*   `128` resolution: `--batch-gpu 8` on V100, `--batch-gpu 16` on A100
*   `1024` resolution: `--batch-gpu 1` on V100, `--batch-gpu 4` on A100





In [ ]:
!python /content/stylegan_xl/train.py \
--outdir=/content/drive/MyDrive/stylegan_xl/training-runs/itp-canyons-1024 \
--cfg=stylegan3-t \
--data=/content/drive/MyDrive/stylegan_xl/data/itp-canyons-1024.zip \
--gpus=1 \
--batch=64 \
--mirror=1 \
--snap 8 \
--batch-gpu 1 \
--kimg 3000 \
--syn_layers 10 \
--superres \
--up_factor 2 \
--head_layers 7 \
--freezed 4 \
--path_stem /content/drive/MyDrive/stylegan_xl/training-runs/itp-canyons-512/00000-stylegan3-t-itp-canyons-512-gpus1-batch64/best_model.pkl


# Image Generation

In [ ]:
!python gen_images.py --outdir=/content/out --trunc=0.7 --seeds=0-100 --batch-sz 1 \
  --network=/content/drive/MyDrive/stylegan_xl/training-runs/ladiescrop-512/00000-stylegan3-t-ladiescrop-512-gpus1-batch128/best_model.pkl

In [ ]:
!zip -r ladiescrop-xl-512_100.zip /content/out

# Video Generation

In [ ]:
!python /content/stylegan_xl/gen_images.py --outdir=/content/out --trunc=0.8 --seeds=0-250 --batch-sz 1 \
  --network=/content/drive/MyDrive/stylegan_xl/training-runs/ladiescrop-1024/00000-stylegan3-t-ladiescrop-1024-gpus1-batch128/best_model.pkl

In [ ]:
!zip -r /content/ladiescrop-1024_250samples.zip /content/out


In [ ]:
%cd "/content/stylegan_xl"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- gen_video.py

In [ ]:
!pip install imageio-ffmpeg

In [ ]:
!python /content/stylegan_xl/gen_video.py \
--output=/content/lerp-stabilize-ladiescrop1024.mp4 \
--trunc=0.8 \
--seeds=250,243,229,206,199,198,191,190,250 \
--grid=1x1 \
--w-frames=60 \
--stabilize-video \
--network=/content/drive/MyDrive/stylegan_xl/training-runs/ladiescrop-1024/00000-stylegan3-t-ladiescrop-1024-gpus1-batch128/best_model.pkl

In [ ]:
!python gen_video.py --help

In [ ]:
%cd /content/stylegan_xl